МФТИ ФИВТ: Курс Машинное Обучение (осень, 2016), Арсений Ашуха, ars.ashuha@gmail.com

<h1 align="center">Organization Info</h1> 

**Дополнительный материал для выполнения дз**:
- Hastie, The Elements of Statistical Learning, https://goo.gl/k3wfEU
    - 2.9 Model Selection and the Bias–Variance Tradeoff 
    - 15 Random Forests
- Соколов, Семинары по композиционным методам, https://goo.gl/sn8RyJ
- Andrew Ng, Bias vs. Variance, https://goo.gl/1ISZ6Y

**Оформление дз**: 
- Присылайте выполненное задание на почту ``ml.course.mipt@gmail.com``
- Укажите тему письма в следующем формате ``ML2016_fall <номер_группы> <фамилия>``, к примеру -- ``ML2016_fall 401 ivanov``
- Выполненное дз сохраните в файл ``<фамилия>_<группа>_task<номер>.ipnb``, к примеру -- ``ivanov_401_task1.ipnb``

**Вопросы**:
- Присылайте вопросы на почту ``ml.course.mipt@gmail.com``
- Укажите тему письма в следующем формате ``ML2016_fall Question <Содержание вопроса>``

--------
- **PS1**: Мы используем автоматические фильтры, и просто не найдем ваше дз, если вы не аккуратно его подпишите.
- **PS2**: Напоминаем, что дедлайны жесткие, письма пришедшие после автоматически удаляются =( чтобы соблазна не было 

<h1 align="center">Check Questions</h1> 

Ответе на вопросы своими словами (загугленный материал надо пересказать), ответ обоснуйте (напишите и ОБЪЯСНИТЕ формулки если потребуется), если не выходит, то вернитесь к лекции дополнительным материалам:

**Вопрос 1**: Какие формулы у шума, смещения, разброса? Какой смысл у этих компонент?

<Ответ>

**Вопрос 2**: 4. Приведите пример семейства с маленьким смещением и большим разбросом. Приведите пример семейства с большим смещением и маленьким разбросом.

<Ответ>

**Вопрос 3**: Как сгенерировать подвыборку с помощью бутстрапа?

<Ответ>

**Вопрос 4**: Что такое бэггинг?

<Ответ>

**Вопрос 5**:  Как соотносятся смещение разброс композиции, построенной с помощью бэггинга, со смещением и разбросом одного базового алгоритма?

<Ответ>

**Вопрос 6**: Как обучается случайный лес? В чем отличия от обычной процедуры построения решающих деревьев?

<Ответ>

**Вопрос 7**: Почему хорошими базовыми алгоритмами для бэггинга являются именно деревья?

<Ответ>

**Вопрос 8**: Как оценить качество случайного леса с помощью out-of-bag-процедуры?

<Ответ>

-----------
PS: Если проверяющий не понял ответ на большинство вопросов, то будет пичалька. Пишите так, чтобы можно было разобраться. 

<h1 align="center">Bagging</h1> 

Известно, что бэггинг плохо работает, если в качестве базовых классификаторов взять knn. Попробуем понять причины на простом примере.

Пусть дана выборка $X^l$ из $l$ объектов с ответами из множества $Y = \{−1, +1\}$. Будем рассматривать классификатор одного ближайшего соседа в качестве базового алгоритма. Построим с помощью бэггинга композицию длины $N$:

$$a_N(x) = sign(\sum_{n=1}^{N} b_n(x))$$

Оцените вероятность того, что ответ композиции на произвольном объекте x будет
отличаться от ответа одного классификатора ближайшего соседа, обученного по всей
выборке. Покажите, что эта вероятность стремится к нулю при N → ∞.

-----
**<Решение>**

<h1 align="center">Bagging Implementation</h1> 

Реализуйте беггинг.

In [57]:
np.array([[1,2,3],[1,2,3], [1,2,3]])[np.ix_([0,1, 2], [1,2])]

array([[2, 3],
       [2, 3],
       [2, 3]])

In [174]:
import warnings

import numpy as np
from copy import deepcopy
from sklearn.base import ClassifierMixin, BaseEstimator
from numpy.random import choice

warnings.simplefilter("ignore")


class BaggingClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, base_estimator, n_estimators, items_rate=1.0, features_rate=1.0):
        """
        Parameters
        ----------
        base_estimator: sklearn.Classifier
            Базовый алгоритм, который можно обучить (есть метод fit).
            Для обучение композиции нужно много таких, можэно получить с помощю copy.deepcopy

        n_estimators: int
            Число алгоритмов в композиции

        items_rate: float > 0
            Доля объектов из трейна, на которой будет обучаться каждый базовый алгоритм

        features_rate: float > 0
            Доля фичей, на которой будет обучаться и применяться каждый базовый алгоритм
        """
        self.base_estimator = base_estimator
        self.n_estimators = n_estimators
        self.items_rate = items_rate
        self.features_rate = features_rate
        
        self.trained = False

    def fit(self, X, y):
        """
        Метод должен обучить композицию алгоритмов, используя X, y как обучающую выборку.
        Не забудте реализорвать функционал выбора случайных объектов и фичей.

        Parameters
        ----------
        X: 2d np.array
        y: 1d np.array
        """
        # Тут храните обученные базовые алгоритмы
        self.estimators = []
        # Тут храните фичи для каждого алгоритма
        self.features_idx = []
        
        for i in range(self.n_estimators):
            estimator = deepcopy(self.base_estimator)
            # =======================================
            # Обучите базовые алгоритмы
            # =======================================
            
            objects_subsample_size = (X.shape[0]) * self.items_rate
            features_subset_size = (X.shape[1]) * self.features_rate

            object_indecies = choice(X.shape[0], size=objects_subsample_size, replace=True)
            feature_indecies = choice(X.shape[1], size=features_subset_size, replace=True)
            
            self.features_idx.append(feature_indecies)
        
            estimator.fit(X[np.ix_(object_indecies, feature_indecies)], y)
            self.estimators.append(estimator)
            
        self.trained = True
        return self

    def predict(self, X):
        """

        Parameters
        ----------
        X: 2d np.array матрица объекты признаки на которых нужно сказать ответ

        Returns
        -------
        y_pred: 1d np.array, Вектор классов для каждого объекта
        """
        if not self.trained:
            raise Exception('Bagging classifier instance is not fitted yet')
        
        probs = [] # Храните тут ответы каждого базового алгоритма
        
        for i in range(self.n_estimators):
            # =======================================
            # Получите ответы (вероятности) от всех базовых алгоритмов
            # ======================================
            probs.append(self.estimators[i].predict_proba(X[:, self.features_idx[i]]))
        
        # =======================================
        # Усредните вероятности полученные от базовых алгоритмов
        # =======================================
        
        y_pred = np.argmax((np.mean(probs, axis=0)), axis=1)

        return y_pred

### Titanic Test

In [175]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

titanic = pd.read_csv('./data/train.csv')[['Survived', 'Pclass', 'Sex', 'Age', 'Fare']]

sex_encoder = LabelEncoder()
titanic.Sex = sex_encoder.fit_transform(titanic.Sex)
features = ['Pclass', 'Sex', 'Age', 'Fare']

In [176]:
X, y = titanic[features].values, titanic.Survived.values
X = np.nan_to_num(X)
X_train, y_train, X_test, y_test = X[:500], y[:500], X[500:], y[500:]

Нужно обучить свой беггинг на датасете титаник, и посмотреть работает ли он. 

In [177]:
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

In [178]:
# =======================================
# Обучите беггинг над DecisionTreeClassifier с 10 моделями
# =======================================
tree_estimator = DecisionTreeClassifier()
clf = BaggingClassifier(base_estimator=tree_estimator, n_estimators=10).fit(X_train, y_train)
print accuracy_score(clf.predict(X_train), y_train), accuracy_score(clf.predict(X_test), y_test)

0.586 0.626598465473


Проведите эксперименты:
    - Работает-ли беггинг лучше чем просто линейная модель?
    - Какой items_rate и features_rate работает лучше и почему?

In [179]:
# =======================================
# Обучите беггинг над DecisionTreeClassifier с 100 моделями
# =======================================
clf = BaggingClassifier(base_estimator=tree_estimator, n_estimators=100).fit(X_train, y_train)
acc = accuracy_score(clf.predict(X_train), y_train), accuracy_score(clf.predict(X_test), y_test)
acc

(0.61599999999999999, 0.61892583120204603)

In [180]:
from sklearn.linear_model import LogisticRegression
# =======================================
# Обучите LogsiticRegression 
# =======================================
clf = LogisticRegression(n_jobs=-1).fit(X_train, y_train)
acc = accuracy_score(clf.predict(X_train), y_train), accuracy_score(clf.predict(X_test), y_test)
acc

(0.80000000000000004, 0.76982097186700771)

### Adult Test

In [181]:
adult = pd.read_csv(
    './data/adult.data', 
    names=[
        "Age", "Workclass", "fnlwgt", "Education", "Education-Num", "Martial Status",
        "Occupation", "Relationship", "Race", "Sex", "Capital Gain", "Capital Loss",
        "Hours per week", "Country", "Target"], 
    header=None, na_values="?")

adult = pd.get_dummies(adult)
adult["Target"] = adult["Target_ >50K"]
X, y = adult[adult.columns[:-3]].values, adult[adult.columns[-1]].values
X_train, y_train, X_test, y_test = X[:20000], y[:20000], X[20000:], y[20000:]

IOError: File ./data/adult.data does not exist

Ответте на вопросы:
    - Работает-ли беггинг лучше чем просто линейная модель?
    - Какой items_rate и features_rate работает лучше и почему?

In [ ]:
...

<h1 align="center">Text, Image Classification</h1> 

Дальше в каждом эксперименте нужно: 
- сравниться с линейной моделью ( какую лучше выбрать?=) )
- сделать выбор в пользу одной из моделей
- выбор обосновать, почему одна из моделей хуже а другая лучше
- что такое хуже и лучше
- попробуйте беггинг над деревьями и линейными моделями 
- почему работает или не работает, какие особенности данных на это влияют

### Text classification

In [168]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')

In [169]:
vectorizer = TfidfVectorizer()
X_train, y_train = vectorizer.fit_transform(newsgroups_train.data), newsgroups_train.target
X_test,  y_test  = vectorizer.transform(newsgroups_test.data), newsgroups_test.target

In [184]:
%%time
# =======================================
# Обучите Линейную модель 
# =======================================
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression().fit(X_train, y_train)
print accuracy_score(clf.predict(X_train), y_train), accuracy_score(clf.predict(X_test), y_test)

0.8 0.769820971867
CPU times: user 10 ms, sys: 0 ns, total: 10 ms
Wall time: 6.82 ms


In [186]:
%%time
# =======================================
# Обучите беггинг над DecisionTreeClassifier
# =======================================

clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=100).fit(X_train, y_train)
print accuracy_score(clf.predict(X_train), y_train), accuracy_score(clf.predict(X_test), y_test)

0.614 0.616368286445
CPU times: user 167 ms, sys: 0 ns, total: 167 ms
Wall time: 167 ms


### Image classification

In [187]:
from utils import load_cifar10
X_train, y_train, X_val, y_val, X_test, y_test = load_cifar10('./data/cifar10')[0]

ImportError: No module named utils

In [188]:
X_train, X_test = X_train.reshape(X_train.shape[0], -1), X_test.reshape(X_test.shape[0], -1)

In [ ]:
%%time
# =======================================
# Обучите Линейную модель 
# =======================================z
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression().fit(X_train, y_train)
clf.predict(X_test)

In [ ]:
%%time
# =======================================
# Обучите беггинг над DecisionTreeClassifier
# =======================================
clf = BaggingClassifier(base_estimator=DecisionTreeClassifier(), n).

<h1 align="center">Random Forest Feature Impotance</h1> 

Опишите как вычисляется важность фичей в дереве, можите изучить как работает  feature\_importances_ в sklearn.

---

<Ответ>

Почитайте Feature Impotance для Adult и Titanic (используйте полный датасет), ПРОИНТЕРПРЕТИРУЙТЕ резульататы.

### Adult

In [ ]:
X, y = adult[adult.columns[:-3]].values, adult[adult.columns[-1]].values
X_train, y_train, X_test, y_test = X[:20000], y[:20000], X[20000:], y[20000:]

In [ ]:
clf = BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=100, 
                        items_rate=1, features_rate=1).fit(X_train, y_train)

In [ ]:
# =======================================
# Посчитайте feature_importances для clf
# =======================================

### Titanick

In [164]:
X, y = titanic[features].values, titanic.Survived.values
X = np.nan_to_num(X)
X_train, y_train, X_test, y_test = X[:500], y[:500], X[500:], y[500:]

In [165]:
clf = BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=100, 
                        items_rate=1, features_rate=1).fit(X_train, y_train)

In [166]:
# =======================================
# Посчитайте feature_importances для clf
# =======================================

In [167]:
clf.feature_importances_

AttributeError: 'BaggingClassifier' object has no attribute 'feature_importances_'